In [1]:
import sys
import pandas as pd
import os
import re
import glob
import shutil
import numpy as np
#import matplotlib.pyplot as plt
#import skimage
from pathlib import Path
import shutil
from base64 import b64encode
from base64 import b64decode

#plt.rcParams["figure.figsize"] = (10,10)

from IPython.core.display import display, HTML
# make the Jupyter notebook use the full screen width
display(HTML("<style>.container { width:99% !important; }</style>"))

In [2]:
PATH="/notebook/nas-trainings/arne/DGFISMA/DATA/"

In [3]:
import random
random.seed(10)

definitions=open( os.path.join( PATH , "definition_extraction/bootstrapped_training_set/iteration_1/training_set_def" ) ).read().strip("\n").split("\n")
no_definitions=open( os.path.join( PATH , "definition_extraction/bootstrapped_training_set/iteration_1/training_set_no_def" ) ).read().strip("\n").split("\n")

training_set=[]

for definition in definitions:
    training_set.append( (definition, 1 ) )

for no_definition in no_definitions:
    training_set.append( (  no_definition, 0 ) )
        
random.shuffle( training_set )

with open( os.path.join(  PATH ,  "definition_extraction/bootstrapped_training_set/iteration_1/train_dgfisma.csv" ) , "w" ) as f:
    for text, label in training_set:
        if label==1:
            f.write( f'"{text}"⚫definition⚫{label}\n'   )
        else:
            f.write( f'"{text}"⚫no_definition⚫{label}\n'   )


In [4]:
#https://github.com/YipingNUS/DefMiner/blob/master/corpus/wcl_datasets/annotated/wiki_combined.txt

training_set=[]

wikipedia_def=open( os.path.join( PATH, "definition_extraction/bootstrapped_training_set/wikipedia_definitions" )    ).read().strip("\n").split("\n")
for sentence in wikipedia_def:
    if "/DEF" in sentence:
        sentence=sentence.replace( "/TERM", "").replace( "/DEF", "" ).replace( "/O", "" ).replace( "-LBR-" , "(" ).replace( "-RBR-", ")" ).replace( "&amp", "&").replace( "&nbsp", " ")
        training_set.append( ( sentence, 1  )   )
    else:
        training_set.append( ( sentence, 0  )   )
        
with open( os.path.join( PATH, "definition_extraction/bootstrapped_training_set/iteration_1/train_wiki.csv"   )  , "w" ) as f:
    for text, label in training_set:
        if label==1:
            f.write( f'"{text}"⚫definition⚫{label}\n'   )
        else:
            f.write( f'"{text}"⚫no_definition⚫{label}\n'   )      

In [5]:
!cat /notebook/nas-trainings/arne/DGFISMA/DATA/definition_extraction/bootstrapped_training_set/iteration_1/train_dgfisma.csv /notebook/nas-trainings/arne/DGFISMA/DATA/definition_extraction/bootstrapped_training_set/iteration_1/train_wiki.csv > /notebook/nas-trainings/arne/DGFISMA/DATA/definition_extraction/bootstrapped_training_set/iteration_1/train_dgfisma_wiki.csv

In [6]:
!python ../bert_classifier/src/train.py  \
--filename "/notebook/nas-trainings/arne/DGFISMA/DATA/definition_extraction/bootstrapped_training_set/iteration_1/train_dgfisma_wiki.csv"   \
--model_storage_directory '../bert_classifier/models_dgfisma_def_extraction' \
--device "cuda:0" \
--train_bert "last_layer" \
--epochs 10 \
--delimiter ⚫

2020-06-30 13:06:54.994096: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-06-30 13:06:54.994182: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-06-30 13:06:54.994191: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Using TensorFlow backend.
Lock 140282263617208 acquired on /root/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c6

Epoch:  80%|█████████████████████████████▌       | 8/10 [05:03<01:16, 38.30s/it]epoch: 9
Train loss: 0.01849788731958409
Validation Accuracy: 0.9742647058823529


Epoch:  90%|█████████████████████████████████▎   | 9/10 [05:42<00:38, 38.43s/it]epoch: 10
Train loss: 0.016914490617363526
Validation Accuracy: 0.9751838235294118


Epoch: 100%|████████████████████████████████████| 10/10 [06:20<00:00, 38.09s/it]
saving model in ../bert_classifier/models_dgfisma_def_extraction/run_2020_06_30_13_06_55_11f36c372745/distilbert-base-uncased_model_10.pth
Configuration saved in ../bert_classifier/models_dgfisma_def_extraction/run_2020_06_30_13_06_55_11f36c372745/config.json


In [7]:
import time
from sklearn.metrics import classification_report

start=time.time()

!python ../bert_classifier/src/predict.py  \
--filename "/notebook/nas-trainings/arne/DGFISMA/DATA/definition_extraction/bootstrapped_training_set/iteration_1/test_sentences"  \
--model_path "../bert_classifier/models_dgfisma_def_extraction/run_2020_06_30_13_06_55_11f36c372745/distilbert-base-uncased_model_10.pth" \
--device "cuda:0"  \
--output_file "/notebook/nas-trainings/arne/DGFISMA/DATA/definition_extraction/bootstrapped_training_set/iteration_1/test_sentences_predict" \
--batch_size 32

end=time.time()
print( end-start )

test_sentences_predict=open( "/notebook/nas-trainings/arne/DGFISMA/DATA/definition_extraction/bootstrapped_training_set/iteration_1/test_sentences_predict"  ).read().strip( "\n" ).split( "\n" )
test_sentences_predict = [ int(label.split()[0]) for label in test_sentences_predict ]

test_labels=open( "/notebook/nas-trainings/arne/DGFISMA/DATA/definition_extraction/bootstrapped_training_set/iteration_1/test_labels"  ).read().strip( "\n" ).split( "\n" )
test_labels=[ int(label)  for label in test_labels  ]

print( classification_report( test_labels, test_sentences_predict   )  )


2020-06-30 13:18:16.593421: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-06-30 13:18:16.593505: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2020-06-30 13:18:16.593517: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Using TensorFlow backend.
loading ../bert_classifier/models_dgfisma_def_extraction/run_2020_06_30_13_06_55_11f36c372745/distilbert-base-uncased_model_10.

#Permformance when using Fasttext (on the same test set):

( see http://localhost:8005/notebooks/nas-trainings/arne/DGFISMA/definition_extraction/notebooks/test_fasttext_classifier.ipynb)

Use this dockerfile:

/nas-trainings/arne/DGFISMA/definition_extraction/Dockerfile


             precision    recall  f1-score   support

           0       0.74      0.86      0.79        29
           1       0.83      0.69      0.75        29

   micro avg       0.78      0.78      0.78        58 <br/>
   macro avg       0.78      0.78      0.77        58 <br/>
weighted avg       0.78      0.78      0.77        58 <br/>

In [ ]:
 0       0.88      1.00      0.94        29
           1       1.00      0.86      0.93        29

   micro avg       0.93      0.93      0.93        58
   macro avg       0.94      0.93      0.93        58
weighted avg       0.94      0.93      0.93        58

In [ ]:
#!pip install bs4

In [181]:
import requests
from cleaning import clean_html, delete_annexes

http="https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=CELEX:32015R0063"

http="https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=celex:32018R0815"


http="https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=celex:52018SC0073"

http= "https://eur-lex.europa.eu/legal-content/EN/ALL/?uri=celex:52017PC0790"

page_response=requests.get( http )

articles=clean_html( page_response.content.decode( 'utf-8' ) )
articles=delete_annexes(articles  )
articles="\n".join(articles)

with open( os.path.join(  "/notebook/nas-trainings/arne/DGFISMA/definition_extraction/notebooks/bootstrapped_training_set/iteration_1/", "eurlex_text_celex:52017PC0790" ) , "w" ) as f:
        f.write(  articles  )


In [ ]:
#open an Eurlex document